In [3]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import join
import csv
import sys
import jieba
import jieba.posseg as pseg
import re
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
print("【訊息】目前的工作目錄:",os.getcwd())
import math

【訊息】目前的工作目錄: /workspace/Randy/17life


In [120]:
csv.field_size_limit(int(sys.maxsize/10000000000))
df=pd.read_csv('./data/deal_final_0605', sep=',', engine='python', encoding='utf-8-sig')

刪除nan的itemName行數 並重置索引

In [107]:
# nanList = list(df['item_name'][df['item_name'].isnull()].index)
# df = df.drop(df.index[nanList])
# df['item_name'][df['item_name'].isnull()]
# df = df.reset_index()

In [121]:
itemName = df['item_name']
symbols = r"[！!\-(－（)）.（+/，#_;:=】？?《》|【\]\[~★♥~↘]"
tmp = [re.sub(symbols," ",str(title)) for title in itemName ]
symbols2 = r"[ \d]+元|[\d.]+折[起]?|\$[ \d]+"
itemNameC = [re.sub(symbols2," ",title) for title in tmp ]

In [130]:
itemNameC = itemNameC[:100]

In [141]:
itemNameToken = [ list(filter(lambda x:x.word!=" ",pseg.lcut(i))) for i in itemNameC ]

In [218]:
itemNameTokenN =[ list(filter(lambda x:x.flag=="n",i)) for i in itemNameToken ]
itemNameTokenN = [ list(map(lambda x: x.word,i)) for i in itemNameTokenN ]

In [219]:
df_codetest = list(np.int32(df.code[:100]))

In [238]:
df_codetest= np.array(df_codetest).reshape(-1,1)
itemNameTokenN = np.array(itemNameTokenN).reshape(-1,1)
tmp = np.concatenate((df_codetest,itemNameTokenN), axis=1)
code_token = pd.DataFrame(tmp,columns=['code', 'tokens'])